In [0]:
import pandas as pd
import numpy as np
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')

In [0]:
from sklearn.model_selection import train_test_split                                      #to split the dataset for training and testing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [0]:
path_1 = "/content/drive/My Drive/BERT_Anadi/Data Produced/bert_train_embed_1000_docs.pickle"
path_2 = "/content/drive/My Drive/BERT_Anadi/Data Produced/bert_gs_embed_all.pickle"


data = pd.read_pickle((path_1))
# data = data.drop(columns = ['doc_id'])
X_pred = (pd.read_pickle(path_2))

# gc.collect()

In [0]:
data.shape

(130759, 1026)

In [0]:
X_pred.shape

(9092, 1025)

In [0]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import train_test_split                                      #to split the dataset for training and testing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

target = data.importance         
data.drop(['importance', 'doc_id'], axis=1, inplace=True)     # taking the training data features
                                 
# del data

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, random_state = 32) # in this our main data is split into train and test

del data, target
gc.collect()

0

In [0]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(91531, 1024) (91531,) (39228, 1024) (39228,)


In [0]:
X_pred.drop(columns = ['doc_id'], inplace = True)
X_pred.shape

(9092, 1024)

In [0]:
model = XGBClassifier(tree_menthod = 'gpu_hist')
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, tree_menthod='gpu_hist', verbosity=1)

In [0]:
pred = model.predict(X_test)
f1_score(pred, y_test)

0.12521853146853146

In [0]:
cutoff = np.arange(0.01, 0.99, 0.01)
f1 = []
acc = []
pred = (model.predict_proba(X_test))[:,1]

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))
    acc.append(accuracy_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=cutoff, y=f1,
                    mode='lines',
                    name='f1 score'))
fig.add_trace(go.Scatter(x=cutoff, y=acc,
                    mode='lines',
                    name='accuracy'))
fig.show()

max_cutoff = cutoff[f1 == max(f1)]

In [0]:
params = {'max_depth': 4,
              'gamma': 0.5,
              'learning_rate':0.05,
              'subsample': 0.8,
              'colsample_bytree': 0.8,
              #'min_child_weight' : int(min_child_weight),
              'eta': 0.05,
              'eval_metric': 'auc'}

In [0]:
#Initializing an XGBClassifier with default parameters and fitting the training data
from xgboost import XGBClassifier
classifier1 = XGBClassifier(**params).fit(X_train, y_train)

In [0]:
 pred = classifier1.predict(X_test)
f1_score(pred, y_test)

0.1017516456543568

In [0]:
cutoff = np.arange(0.01, 0.99, 0.01)
f1 = []
acc = []
pred = (classifier1.predict_proba(X_test))[:,1]

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))
    acc.append(accuracy_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=cutoff, y=f1,
                    mode='lines',
                    name='f1 score'))
fig.add_trace(go.Scatter(x=cutoff, y=acc,
                    mode='lines',
                    name='accuracy'))
fig.show()

max_cutoff = cutoff[f1 == max(f1)]

In [0]:
model = XGBClassifier(tree_menthod = 'gpu_hist')
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, tree_menthod='gpu_hist', verbosity=1)

In [0]:
pred = model.predict(X_test)
f1_score(pred, y_test)

0.12521853146853146

In [0]:
cutoff = np.arange(0.01, 0.99, 0.01)
f1 = []
acc = []
pred = (model.predict_proba(X_test))[:,1]

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))
    acc.append(accuracy_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=cutoff, y=f1,
                    mode='lines',
                    name='f1 score'))
fig.add_trace(go.Scatter(x=cutoff, y=acc,
                    mode='lines',
                    name='accuracy'))
fig.show()

max_cutoff = cutoff[f1 == max(f1)]

In [0]:
 #Importing necessary libraries
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.metrics import mean_squared_error

#Converting the dataframe into XGBoost’s Dmatrix object
dtrain = xgb.DMatrix(X_train, label=y_train)

#Bayesian Optimization function for xgboost
#specify the parameters you want to tune as keyword arguments
def bo_tune_xgb(max_depth, gamma, n_estimators ,learning_rate):
     params = {'max_depth': int(max_depth),
              'gamma': gamma,
              'n_estimators': int(n_estimators),
              'learning_rate':learning_rate,
              'subsample': 0.8,
              'colsample_bytree': 0.8,
              #'min_child_weight' : int(min_child_weight),
              'eta': 0.05,
              'eval_metric': 'auc'}
    #Cross validating with the specified parameters in 5 folds and 70 iterations 
     cv_result = xgb.cv(params, dtrain, num_boost_round=70, nfold=5)
    #Return the negative RMSE
     return cv_result['test-auc-mean'].iloc[-1]

#Invoking the Bayesian Optimizer with the specified parameters to tune
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (3, 10),
                                             'gamma': (0, 1),
                                             'learning_rate':(0.05,0.3),
                                             'n_estimators':(100,120)
 #'min_child_weight':(1,10)
                                            })

#performing Bayesian optimization for 5 iterations with 8 steps of random exploration with an #acquisition function of expected improvement
xgb_bo.maximize(n_iter=2, init_points=3, acq='ei')

|   iter    |  target   |   gamma   | learni... | max_depth | n_esti... |
-------------------------------------------------------------------------


In [0]:
params = {'max_depth': 4,
              'gamma': 0.5,
              'learning_rate':0.05,
              'subsample': 0.8,
              'colsample_bytree': 0.8,
              #'min_child_weight' : int(min_child_weight),
              'eta': 0.05,
              'eval_metric': 'auc'}

In [0]:
#Initializing an XGBClassifier with default parameters and fitting the training data
from xgboost import XGBClassifier
classifier1 = XGBClassifier(**params).fit(X_train, y_train)

In [0]:
 pred = classifier1.predict(X_test)
f1_score(pred, y_test)

0.1017516456543568

In [0]:
cutoff = np.arange(0.01, 0.99, 0.01)
f1 = []
acc = []
pred = (classifier1.predict_proba(X_test))[:,1]

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))
    acc.append(accuracy_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=cutoff, y=f1,
                    mode='lines',
                    name='f1 score'))
fig.add_trace(go.Scatter(x=cutoff, y=acc,
                    mode='lines',
                    name='accuracy'))
fig.show()

max_cutoff = cutoff[f1 == max(f1)]